**Librerías y DF**

In [2]:
import pandas as pd 
import numpy as np
import re
df= pd.read_csv(r"C:\Users\pablo\Documents\GitHub\WhiteHosting\CSVs\Valencia\Airbnb Valencia (sin limpiar).csv")

**Limpieza**

In [3]:
#Columna ID a str
df["ID"] = (df["ID"]).astype(str)

#Quitamos el € de la columna de precios
df["Precio por noche Octubre"]= df["Precio por noche Octubre"].apply(lambda x: x.replace("€","").replace(".","").strip())
df["Precio por noche Agosto"] = df["Precio por noche Agosto"].apply(lambda x: x.replace("€","").replace(".","").strip())

#Cambiamos el precio no disponible por 0, y convertirlos a enteros
def cambiar_a_nulos(x):
    if x=="No disponible en Octubre" or x=="No disponible en Agosto":
        return np.nan
    else:
        return x
df["Precio por noche Octubre"]= df["Precio por noche Octubre"].apply(cambiar_a_nulos).astype('Int64')
df["Precio por noche Agosto"] = df["Precio por noche Agosto"].apply(cambiar_a_nulos).astype('Int64')

#Limpiamos el texto de camas, dormitorios, numero de valoraciones y viajeros
def solo_numero(x):
    try:
        return int(x.split()[0])
    except (ValueError, AttributeError):
        return np.nan
df["Camas"]= df["Camas"].apply(solo_numero).astype('Int64')
df["Viajeros máximos"]= df["Viajeros máximos"].apply(solo_numero).astype('Int64')
df["Dormitorios"]= df["Dormitorios"].apply(solo_numero).astype('Int64')
df["Numero valoraciones"]= df["Numero valoraciones"].apply(solo_numero).astype('Int64')

#Limpiamos la columna Valoracion   
df['Valoración'] = df['Valoración'].replace('Sin valoraciones', np.nan).replace('Sin evaluaciones', np.nan).replace('Nuevo', np.nan).str.replace(',', '.').astype(float)

#Limpiamos la columna del Baño, dejándomo como minimo en 1
def solo_numero_baño(x):
    try:
        return int(x.split()[0])
    except (ValueError, AttributeError):
        return 1
df["Baños"]= df["Baños"].apply(solo_numero_baño).astype('Int64')
#Columna de tiología
df['Tipología'] = df['Tipo de habitación'].str.split().str[:1]
df["Tipología"]= df["Tipología"].astype(str).apply(lambda x: x.replace("['","").replace("']","").strip())

#Añadimos columna de Precio medio
df['Precio Medio'] = df[['Precio por noche Agosto', 'Precio por noche Octubre']].mean(axis=1)
df['Precio Medio'] = df['Precio Medio'].round().astype("Int64")

#Añadir una nueva columna Título
df['Título'] = df['Tipo de habitación'].str.split(':| en ', n=1).str[0]
df['Título'] = df['Título'].str.strip()

#Añadir una nueva columna "Municipio"
df['Municipio'] = df['Tipo de habitación'].str.split(',').str[-2].str.strip()
df['Municipio'] = df['Municipio'].str.split(' en ', n=1).str[-1]
df['Municipio'] = df['Municipio'].str.split(' en ', n=1).str[-1]
df['Municipio'] = df['Municipio'].replace('·','')
df['Municipio'] = df['Municipio'].replace(regex=r'[^a-zA-Z0-9\s]', value='')

#Movemos las columnas para mejor visualización
df = df[['ID', 'Tipo de habitación', 'Título', 'Tipología', 'Ciudad', 'Localización', 'Municipio', 'Precio por noche Agosto', 'Precio por noche Octubre', 'Precio Medio', 'Valoración', 'Numero valoraciones', 'Dormitorios', 'Baños', 'Camas', 'Viajeros máximos']]

#Rellenamos los ID´s con zeros delante para tenerlos igualmente largos.
df['ID'] = df['ID'].astype(str).str.zfill(20)


**Exportar en CSV**

In [4]:
df.to_csv("Airbnb Valencia Limpio.csv", index=False)